# Tutorial: Elasticity
In this tutorial we demonstrate the use of the compute_elasticity and compute_stress_analysis functions. 

In [ ]:
# Run this line only the first time you open a tutorial in Google Colab
pip install 'git+https://github.com/nasa/puma.git'

Let's start by importing pumapy, Numpy and Scipy ndimage.

In [1]:
import os
import pumapy as puma
import numpy as np
import scipy.ndimage as nd

Now, we will run four different verification cases. To set the path of the file outputs (which can be plotted in Paraview), change the following variable:

In [6]:
pumadir = "../../python"
export_path = pumadir + "/tests/out/"

### Verification #1: harmonic averaging, in series along x periodic sides

In [7]:
export_name = 'halfmat'
X = 20
Y = 20
Z = 20
ws = puma.Workspace.from_shape_value((X, Y, Z), 1)
ws[int(X / 2):] = 2
# ws.show_matrix()

elast_map = puma.ElasticityMap()
elast_map.add_isotropic_material((1, 1), 200, 0.3)
elast_map.add_isotropic_material((2, 2), 400, 0.1)

_, u, s, t = puma.compute_elasticity(ws, elast_map, 'x', 'f', solver_type="direct")
puma.export_vti(export_path + export_name, {"ws": ws, "disp": u, "sigma": s, "tau": t})

Initializing and padding domains ... Done
Assembling b vector ... Done
Initializing large data structures ... Done
Assembling A matrix ... 100.0% Done
Solving Ax=b system ... Direct solver ... Done
Computing stresses ... 100.0% Done
Exporting ../../python/tests/out/halfmat.vti ... Done


True

### Verification #2: full built-in beam

In [ ]:
export_name = 'builtinbeam'
X = 10
Y = 50
Z = 10
ws = puma.Workspace.from_shape_value((X, Y, Z), 1)
ws.voxel_length = 1

elast_map = puma.ElasticityMap()
elast_map.add_isotropic_material((1, 1), 200, 0.3)

bc = puma.ElasticityBC.from_workspace(ws)
bc[:, 0] = 0
bc[:, -1, :, :2] = 0
bc[:, -1, :, 2] = -1  # z displacement of -1
# puma.Workspace.show_orientation(bc)

u, s, t = puma.compute_stress_analysis(ws, elast_map, bc, side_bc='f', solver_type="direct")
puma.export_vti(export_path + export_name, {"ws": ws, "disp": u, "sigma": s, "tau": t})

### Verification #3: plate with a hole

In [ ]:
export_name = 'platehole'
X = 50
Y = 50
Z = 3
ws = puma.Workspace.from_shape_value((X, Y, Z), 1)
ws.voxel_length = 1

mask = np.ones((X, Y, Z), dtype=bool)
mask[X//2, Y//2] = 0
distance_mask = nd.morphology.distance_transform_edt(mask)
max_distance = np.max(distance_mask)
distance_mask_display = (distance_mask*255./max_distance).astype(dtype=np.uint8)
in_range = distance_mask <= 17
ws[in_range] = 0

elast_map = puma.ElasticityMap()
elast_map.add_isotropic_material((1, 1), 200, 0.3)

bc = puma.ElasticityBC.from_workspace(ws)
bc[0, :, :, 0] = 0
bc[-1, :, :, 0] = 1

u, s, t = puma.compute_stress_analysis(ws, elast_map, bc, side_bc='f', solver_type="direct")
puma.export_vti(export_path + export_name, {"ws": ws, "disp": u, "sigma": s, "tau": t})

### Verification #4: cracked plate

In [ ]:
export_name = 'crackedplate'
X = 25
Y = 100
Z = 3
ws = puma.Workspace.from_shape_value((X, Y, Z), 1)
ws.voxel_length = 1

ws[:10, Y//2-1:Y//2+1] = 0

elast_map = puma.ElasticityMap()
elast_map.add_isotropic_material((1, 1), 200, 0.3)

bc = puma.ElasticityBC.from_workspace(ws)
bc[:, 0, :, 1] = 0
bc[:, -1, :, 1] = 1

u, s, t = puma.compute_stress_analysis(ws, elast_map, bc, side_bc='f', solver_type="direct")
puma.export_vti(export_path + export_name, {"ws": ws, "disp": u, "sigma": s, "tau": t})